In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

import matplotlib.pyplot as plt

In [2]:
spark = SparkSession.builder.appName('RideSharingServiceAnalysis').getOrCreate()

In [3]:
ride_data = spark.read.option('header', 'true').csv('ride_data.csv')

In [4]:
ride_data.createOrReplaceTempView('ride_data_table')

In [5]:
ride_data.show()

+------+--------+----------+----------------+----------------+--------------+---------------+-----------+----+
|rideID|driverID|customerID|       startTime|         endTime|pickupLocation|dropoffLocation|distance_km|fare|
+------+--------+----------+----------------+----------------+--------------+---------------+-----------+----+
|     1|      25|        12|04-04-2023 07:30|04-04-2023 08:40| Parade Ground|    Dilsuknagar|         25| 300|
|     2|      23|        19|04-04-2023 07:49|04-04-2023 09:00|   S. R. Nagar|    Musheerabad|         20| 240|
|     3|      28|        11|04-04-2023 08:02|04-04-2023 09:13|      Ameerpet|  Chikkadapally|         15| 180|
|     4|      26|        13|04-04-2023 08:27|04-04-2023 09:33|  Narayanaguda|     Panjagutta|         14| 168|
|     5|      27|        18|04-04-2023 08:58|04-04-2017 10:02|      Begumpet|     Kukatpally|         16| 192|
|     6|      22|       110|04-04-2023 09:01|04-04-2023 09:34|    Rasoolpura|           NGRI|         12| 144|
|

In [6]:
customer_data = spark.read.option('header', 'true').csv('customer_data.csv')
customer_data.createOrReplaceTempView('customer_data_table')
customer_data.show()

+----------+-------+----+-------+-------------+
|customerID|  cname|cage|cgender|     location|
+----------+-------+----+-------+-------------+
|        11|   raju|  18|      M|     Ameerpet|
|        12|   rani|  35|      F|Parade Ground|
|        13|   siri|  49|      M| Narayanaguda|
|        14|   hari|  52|      F|     Malakpet|
|        15|madhavi|  22|      F|     Nampally|
|        16|  roopa|  25|      F|  Khairatabad|
|        17|   neha|  31|      F|    Yusufguda|
|        18|  henry|  63|      M|    Begumpeta|
|        19|   guen|  26|      F|  S. R. Nagar|
|       110|  harry|  87|      M|  Rasuoolpura|
+----------+-------+----+-------+-------------+



In [7]:
driver_data = spark.read.option('header', 'true').csv('driver_data.csv')
driver_data.createOrReplaceTempView('driver_data_table')
driver_data.show()

+--------+--------+----+-------+------------------+
|driverID|   dname|dage|dgender|yearsOf Experience|
+--------+--------+----+-------+------------------+
|      21|   raghu|  34|      M|                11|
|      22|   sagar|  25|      M|                 6|
|      23|   kiran|  35|      M|                 9|
|      24|pentayya|  32|      M|                 9|
|      25| kotayya|  31|      M|                 8|
|      26|appparao|  27|      M|                 5|
|      27|    babu|  30|      M|                 7|
|      28|  lokesh|  29|      M|                 6|
|      29|  ramesh|  33|      M|                10|
|     210|  suresh|  26|      M|                 4|
+--------+--------+----+-------+------------------+



In [8]:
longest_ride = spark.sql("select distance_km as longest_distance,pickupLocation,dropoffLocation,startTime,endTime from ride_data_table order by distance_km desc limit 1")
longest_ride.show()

+----------------+--------------+---------------+----------------+----------------+
|longest_distance|pickupLocation|dropoffLocation|       startTime|         endTime|
+----------------+--------------+---------------+----------------+----------------+
|              25| Parade Ground|    Dilsuknagar|04-04-2023 07:30|04-04-2023 08:40|
+----------------+--------------+---------------+----------------+----------------+



In [9]:
shortest_ride = spark.sql("select distance_km as shortest_distance,pickupLocation,dropoffLocation,startTime,endTime from ride_data_table order by distance_km asc limit 1")
shortest_ride.show()

+-----------------+--------------+---------------+----------------+----------------+
|shortest_distance|pickupLocation|dropoffLocation|       startTime|         endTime|
+-----------------+--------------+---------------+----------------+----------------+
|               10|     Yusufguda|   Lakdi-ka-pul|04-04-2023 09:45|04-04-2023 10:39|
+-----------------+--------------+---------------+----------------+----------------+



In [10]:
longest_ride_customer = spark.sql('select ride_data_table.customerID,customer_data_table.cname,ride_data_table.distance_km as longest_distance from ride_data_table inner join customer_data_table on ride_data_table.customerID=customer_data_table.customerID order by ride_data_table.distance_km desc limit 1')
longest_ride_customer.show()

+----------+-----+----------------+
|customerID|cname|longest_distance|
+----------+-----+----------------+
|        12| rani|              25|
+----------+-----+----------------+



In [11]:
shortest_ride_customer = spark.sql('select ride_data_table.customerID,customer_data_table.cname,ride_data_table.distance_km as shortest_distance from ride_data_table inner join customer_data_table on ride_data_table.customerID=customer_data_table.customerID order by ride_data_table.distance_km asc limit 1')
shortest_ride_customer.show()

+----------+-----+-----------------+
|customerID|cname|shortest_distance|
+----------+-----+-----------------+
|        17| neha|               10|
+----------+-----+-----------------+



In [12]:
average_ride_distance = spark.sql('select avg(distance_km) as average_ride_distance from ride_data_table')
average_ride_distance.show()

+---------------------+
|average_ride_distance|
+---------------------+
|                 14.9|
+---------------------+



In [13]:
average_ride_fare = spark.sql('select avg(fare) as average_ride_fare from ride_data_table')
average_ride_fare.show()

+-----------------+
|average_ride_fare|
+-----------------+
|            178.8|
+-----------------+



In [14]:
maximum_ride_fare = spark.sql('select max(fare) as maximum_ride_fare from ride_data_table')
maximum_ride_fare.show()

+-----------------+
|maximum_ride_fare|
+-----------------+
|              300|
+-----------------+



In [15]:
minimum_ride_fare = spark.sql('select min(fare) as minimum_ride_fare from ride_data_table')
minimum_ride_fare.show()

+-----------------+
|minimum_ride_fare|
+-----------------+
|              120|
+-----------------+



In [16]:
driver_count = spark.sql('select driverId,count(driverId) as count from ride_data_table group by driverId order by count desc')
driver_count.createOrReplaceTempView('driver_count_data')
driver_count.show()

+--------+-----+
|driverId|count|
+--------+-----+
|      25|    3|
|      27|    2|
|      29|    1|
|      28|    1|
|      26|    1|
|      22|    1|
|      23|    1|
+--------+-----+



In [17]:
customer_gender = spark.sql('select cgender as customer_gender,count(cgender) as count from customer_data_table group by cgender order by count')
customer_gender.show()

+---------------+-----+
|customer_gender|count|
+---------------+-----+
|              M|    4|
|              F|    6|
+---------------+-----+



In [18]:
driver_gender = spark.sql('select dgender as driver_gender,count(dgender) as count from driver_data_table group by dgender order by count')
driver_gender.show()

+-------------+-----+
|driver_gender|count|
+-------------+-----+
|            M|   10|
+-------------+-----+



# Thank You